In [6]:
import os
import json
import requests
from ruamel.yaml import YAML
import glob
import soundfile as sf
import librosa
import shutil
import re
import sys

BASE_DIR = "/home/khoatlv"
sys.path.append(BASE_DIR)
from Conformer_ASR.scripts.utils import config, Logger, Config, save_manifest

LOGGER = Logger("COLLECTE VIETTEL CUSTOM DATA")
data_config = config.get_config(["collect_viettel_data"]) 
print(data_config)

{'assistant': {'collected': '/home/khoatlv/data/data_collected/viettel/assistant/raw', 'resample': '/home/khoatlv/data/data_collected/viettel/assistant/resample', 'manifests_dir': '/home/khoatlv/data/data_collected/viettel/assistant/manifests'}, 'speeds': [-2, 0], 'speakers': ['hn-quynhanh', 'hcm-diemmy', 'hue-maingoc', 'hn-thanhtung', 'hue-baoquoc', 'hcm-minhquan']}


In [10]:
def collect_viettel_data(data_dir):
    if not os.path.exists(data_dir): os.makedirs(data_dir)
    scripts_path = config.get_config(["lm", "data", "assistant_data"])
    with open(scripts_path, mode='r') as f:
        scripts_data = f.readlines()
        scripts_data = [data.replace("\n", "") for data in scripts_data]
    
    url = "https://viettelgroup.ai/voice/api/tts/v1/rest/syn"
    data = {
        "text": "",
        "voice": "",
        "id": "",
        "without_filter": False,
        "speed": "",
        "tts_return_option": 2
    }

    headers = {
        'Content-type': 'application/json',
        'token': 's8i2XHmgKBQ8Qt1fhZtPf5zOTj-YO5jMakKgGACkCoa7juGNSluuizwAlfdVbzhb'
    }

    num_files = 0
    for script in scripts_data:
        print(f"Start download data for command: {script}")
        for speaker_idx, speaker in enumerate(data_config.speakers):
            for speed_idx, speed in enumerate(data_config.speeds):
                data["text"] = script
                data["voice"] = speaker
                data["speed"] = speed
                
                script_encoded = script.replace(" ", "-")
                wav_id = "_".join([script_encoded, str(speaker_idx), str(speed_idx)]) + ".wav"
                wav_path = os.path.join(data_dir, wav_id)

                if os.path.exists(os.path.join(data_dir, wav_id)):
                    print("\tAlready has this wav: {}".format(wav_id))
                    continue

                response = requests.post(url, data=json.dumps(data), headers=headers)
                if response.status_code == 200:
                    wav_data = response.content
                    f = open(wav_path, "wb")
                    f.write(wav_data)
                    f.close()
                    
                    print("\tDownload successfully wav: {}".format(wav_id))
                else:
                    print("\tHas error when download wav: {}".format(wav_id))


        #         break
        #     break
        # break
    
        num_files += 1
        print(f"Finish download data for command: {script}")
        

def resample_sr(data_dir, resample_dir, manifests_dir):
    wav_paths = glob.glob(os.path.join(data_dir, "*.wav"))

    manifests_data = []
    invalid_data = []
    
    if not os.path.exists(resample_dir): os.mkdir(resample_dir)
    if not os.path.exists(manifests_dir): os.mkdir(manifests_dir)
    
    manifests_path = os.path.join(manifests_dir, "manifests.json")
    # if os.path.exists(resample_dir): os.remove(resample_dir)
    if os.path.exists(manifests_path): os.remove(manifests_path)
    
    for wav in wav_paths:
        _, file_name = os.path.split(wav)
        resample_wav_path = os.path.join(resample_dir, file_name)
        file_name, ext = os.path.splitext(file_name)
        file_name_splited = file_name.split("_")
        
        script = " ".join(file_name_splited[0].split("-"))
        duration = ""
        
        # Convert files
        if os.path.exists(resample_wav_path):
            data, sr = librosa.load(resample_wav_path, sr=16000)
            duration = librosa.get_duration(y=data, sr=sr)
        else:
            try:
                data, sr = sf.read(wav)
                target_sr = 16000
                if sr != target_sr:
                    data = librosa.resample(y=data, orig_sr=sr, target_sr=target_sr)
                    sf.write(resample_wav_path, data, target_sr, subtype='PCM_24')
                else:
                    shutil.copy(wav, resample_wav_path)
                    
                duration = librosa.get_duration(y=data, sr=target_sr)                
            except Exception:
                invalid_data.append(wav)
        
        # write file manifest
        data = {
            "audio_filepath": resample_wav_path,
            "duration": duration,
            "text": script
        }

        manifests_data.append(data)
    
    save_manifest(manifests_path, manifests_data)
    print(invalid_data)
    
# collect_viettel_data(data_config.assistant.collected)
resample_sr(data_config.assistant.collected, data_config.assistant.resample, data_config.assistant.manifests_dir)

[]
